In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from keras_tuner.tuners import RandomSearch

# Function to build the model
def build_model(hp):
    # Load InceptionResNetV2 as the base model
    inception_resnet = InceptionResNetV2(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

    # Build the head model
    headmodel = inception_resnet.output
    headmodel = Flatten()(headmodel)
    headmodel = Dense(hp.Int('units_1', min_value=256, max_value=1024, step=32), activation="relu")(headmodel)
    headmodel = Dense(hp.Int('units_2', min_value=32, max_value=256, step=32), activation="relu")(headmodel)
    headmodel = Dense(4, activation='sigmoid')(headmodel)

    # Build the complete model
    model = Model(inputs=inception_resnet.input, outputs=headmodel)

    # Compile the model
    model.compile(
        optimizer=Adam(learning_rate=hp.Float('learning_rate', min_value=1e-6, max_value=1e-3, sampling='log')),
        loss='mean_squared_error',  # Adjust the loss function based on your specific task

    )

    return model

# Instantiate the tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=7,  # Adjust the number of trials based on your computational resources
    directory='my_tuning_dir',
    project_name='object_detection_tuning'
)

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Perform the hyperparameter search
tuner.search(x=x_train,y=y_train,batch_size=10,epochs=180,
                    validation_data=(x_test,y_test), callbacks=[early_stopping])  # Adjust epochs and validation split as needed

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print(f"Best Hyperparameters: {best_hps}")

# Build the final model with the best hyperparameters
final_model = tuner.hypermodel.build(best_hps)
print("Best Hyperparameters:")
print(f"Units 1: {best_hps.get('units_1')}")
print(f"Units 2: {best_hps.get('units_2')}")
print(f"Learning Rate: {best_hps.get('learning_rate')}")